## 파일 업로드
- https://flask.palletsprojects.com/en/2.2.x/patterns/fileuploads/

## OracleDB

In [1]:
import cx_Oracle
import os

# DB 접속|
def conn_db() :
    os.putenv('NLS_LANG', '.UTF8') # 한글 지원하기    
    conn = cx_Oracle.connect('dct','1234','project-db-stu.ddns.net:1524/xe')
    cursor = conn.cursor()
    return conn, cursor

# DB 접속 해제
def close_db(conn, cursor) :
    cursor.close()
    conn.close()

## 설계도 업로드

In [2]:
from datetime import datetime
UPLOAD_FOLDER = './static/upload'
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}


def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS and filename not in os.listdir(UPLOAD_FOLDER)

def set_filename(filename, a_name) :
    ext = filename.rsplit('.', 1)[1]
    now = datetime.now().strftime('%Y%m%d%H%M%S')
    return f"{a_name}_{now}.{ext}"
    
# 공간 등록
def areaNameCk(M_NO, A_NAME, filename) :
    try :
        conn, cursor = conn_db()  # DB 접속
        A_SRC = set_filename(filename)
        query = f"""INSERT INTO AREA (M_NO, A_NAME, A_SRC) VALUES (:1, :2, :3)"""
        cursor.execute(query, [M_NO, A_NAME, A_SRC])  # 쿼리문 실행 
        conn.commit()
        return True
    except :
        return False

In [ ]:
from flask import Flask, session, request, render_template, redirect, url_for, jsonify, send_from_directory
from werkzeug.utils import secure_filename
from datetime import timedelta
import random


app = Flask(__name__)
app.secret_key = f"I'm_on_TOP!{random.randint(0, 99999):0>5}"
app.config["PERMANENT_SESSION_LIFETIME"] = timedelta(minutes=10) # 로그인 지속 시간
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER


@app.route('/')
def main() :
    return render_template('test.html')

@app.route('/set/map/upload_result', methods=['POST'])
def upload_file():
    if request.method == 'POST':
        # check if the post request has the file part
        session['user_no'] = '17'
        m_no = session['user_no']
        a_name = request.form['A_NAME']
        print(a_name)
        file = request.files['file']
        print(file)
        # If the user does not select a file, the browser submits an
        # empty file without a filename.
        if file.filename == '':
            return jsonify({'message':'파일을 선택해 주십시오.', 'result': False})

        if file and allowed_file(file.filename):
            filename = secure_filename(set_filename(file.filename, a_name))
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            if areaNameCk(m_no, a_name, filename):
                return jsonify({'message':'농장 설계도가 등록되었습니다.', 'result': True})
            else :
                return jsonify({'message':'농장 설계도 등록 실패', 'result': False})
        else :
            return jsonify({'message':'이미지 파일을 선택해 주십시오.(확장자: jpg, png, jpeg)', 'result': False})

    else :
        return jsonify({'message':'로그인이 되어있지 않습니다.', 'result': False})

@app.route('/uploads/<name>')
def download_file(name):
    return send_from_directory(app.config["UPLOAD_FOLDER"], name)

if __name__ == '__main__' :
    app.run(host="192.168.70.63", port=2048)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://192.168.70.63:2048
Press CTRL+C to quit
192.168.70.63 - - [06/Dec/2022 18:02:46] "GET / HTTP/1.1" 200 -
192.168.70.63 - - [06/Dec/2022 18:02:46] "GET /static/bootstrap.bundle.min.js HTTP/1.1" 304 -
192.168.70.63 - - [06/Dec/2022 18:02:46] "GET /static/bootstrap.min.css HTTP/1.1" 304 -
192.168.70.63 - - [06/Dec/2022 18:02:46] "GET /static/bootstrap.css HTTP/1.1" 304 -
192.168.70.63 - - [06/Dec/2022 18:02:46] "GET /static/test.js HTTP/1.1" 304 -
192.168.70.63 - - [06/Dec/2022 18:02:54] "POST /set/map/upload_result HTTP/1.1" 400 -


토마토


In [ ]:
from flask import Flask, flash, session, render_template, redirect, request, url_for
from markupsafe import escape
from datetime import timedelta, datetime
import random
from werkzeug.utils import secure_filename


app = Flask(__name__)
app.secret_key = f"I'm_on_TOP!{random.randint(0, 99999):0>5}"
app.config["PERMANENT_SESSION_LIFETIME"] = timedelta(minutes=60) # 로그인 지속 시간

# 파일 업로드
@app.route('/')
def main() :
    return render_template('test.html')

@app.route('/upload', methods = ['POST'])
def upload_file():
    if request.method == 'POST':
        try:
            f = request.files['file']
            print(f)
            f.save('./static/upload/' + secure_filename(f.filename)) # 파일명을 보호하기 위한 함수, 지정된 경로에 파일 저장
            return render_template('test.html')
        except:
            return render_template('test.html')
    else:
        return render_template('test.html')

if __name__ == '__main__' :
    app.run(host="192.168.70.63", port=2048)